In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_rows', 10000)

In [5]:
# Load the annual country capacity data
annual_data = pd.read_csv("national_generation_capacity_stacked_filtered.csv")
print(np.sort(annual_data.technology.unique()))
# Filter to only technologies of interest -- solar and wind
annual_df = annual_data[
    annual_data.technology.isin(
        [
            "Solar",
            "Wind",
            "Photovoltaic",
            "Concentrated solar power",
            "Differently categorized solar",
            "Differently categorized wind",
            "Renewable energy sources",
        ]
    )
].copy()

['Bioenergy and renewable waste' 'Biomass and biogas' 'Combined cycle'
 'Concentrated solar power' 'Differently categorized fossil fuels'
 'Differently categorized hydro' 'Differently categorized natural gas'
 'Differently categorized renewable energy sources'
 'Differently categorized solar' 'Differently categorized wind'
 'Fossil fuels' 'Gas turbine' 'Geothermal' 'Hard coal' 'Hydro' 'Lignite'
 'Marine' 'Mixed fossil fuels' 'Natural gas' 'Non-renewable waste'
 'Nuclear' 'Offshore' 'Oil' 'Onshore' 'Other and unknown natural gas'
 'Other bioenergy and renewable waste' 'Other fossil fuels'
 'Other or unspecified energy sources' 'Photovoltaics' 'Pumped storage'
 'Pumped storage with natural inflow' 'Renewable energy sources'
 'Reservoir' 'Reservoir including pumped storage' 'Run-of-river'
 'Sewage and landfill gas' 'Solar' 'Total' 'Wind']


In [13]:
# annual_df.groupby(['country','year','technology']).capacity.sum().reset_index()


,country,year,technology,capacity
0,AL,1990,Concentrated solar power,0.000000
1,AL,1990,Differently categorized solar,0.000000
2,AL,1990,Differently categorized wind,0.000000
3,AL,1990,Renewable energy sources,0.000000
4,AL,1990,Solar,0.000000
5,AL,1990,Wind,0.000000
6,AL,1991,Concentrated solar power,0.000000
7,AL,1991,Differently categorized solar,0.000000
8,AL,1991,Differently categorized wind,0.000000
9,AL,1991,Renewable energy sources,0.000000


In [7]:
pv_df = pd.read_pickle('pv_data.pkl')
pv_df.shape

(876480, 5)

In [8]:
pv_df.country.unique()

array(['Spain', 'Austria', 'Portugal', 'Albania', 'Netherlands', 'Italy',
       'Romania', 'Czech Republic', 'Ireland', 'Belgium', 'Switzerland',
       'Germany', 'Great Britain', 'Slovenia', 'Macedonia', 'Greece',
       'Norway', 'Hungary', 'Bulgaria', 'Montenegro'], dtype=object)

In [9]:
wind_df = pd.read_pickle('wind_data.pkl')
wind_df.shape

(745272, 2)

In [10]:
wind_df.head()

,electricity,country
1514764800000,0.310,Spain
1514768400000,0.269,Spain
1514772000000,0.291,Spain
1514775600000,0.224,Spain
1514779200000,0.130,Spain


In [33]:
df2.index = pd.to_datetime(df2.index, unit='ms')
df2 = df2.reset_index().rename(columns={'index':'date_utc_ts'})
df2.head()

/var/folders/s9/9q8cs9l92wd15gsnz7n8f0x40000gn/T/ipykernel_79165/1204736522.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df2.index = pd.to_datetime(df2.index, unit='ms')


,date_utc_ts,electricity,irradiance_direct,irradiance_diffuse,temperature,country
0,2019-01-01 00:00:00,0.0,0.0,0.0,4.024,Spain
1,2019-01-01 01:00:00,0.0,0.0,0.0,3.084,Spain
2,2019-01-01 02:00:00,0.0,0.0,0.0,2.221,Spain
3,2019-01-01 03:00:00,0.0,0.0,0.0,1.384,Spain
4,2019-01-01 04:00:00,0.0,0.0,0.0,0.727,Spain


In [35]:
time_data = pd.read_csv("time_series_60min_singleindex_filtered.csv")
time_data.head()

,utc_timestamp,cet_cest_timestamp,AT_load_actual_entsoe_transparency,AT_load_forecast_entsoe_transparency,AT_price_day_ahead,AT_solar_generation_actual,AT_wind_onshore_generation_actual,BE_load_actual_entsoe_transparency,BE_load_forecast_entsoe_transparency,BE_solar_generation_actual,...,SI_load_actual_entsoe_transparency,SI_load_forecast_entsoe_transparency,SI_solar_generation_actual,SI_wind_onshore_generation_actual,SK_load_actual_entsoe_transparency,SK_load_forecast_entsoe_transparency,SK_solar_generation_actual,SK_wind_onshore_generation_actual,UA_load_actual_entsoe_transparency,UA_load_forecast_entsoe_transparency
0,2014-12-31T23:00:00Z,2015-01-01T00:00:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01T00:00:00Z,2015-01-01T01:00:00+0100,5946.0,6701.0,35.0,NaN,69.0,9484.0,9897.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01T01:00:00Z,2015-01-01T02:00:00+0100,5726.0,6593.0,45.0,NaN,64.0,9152.0,9521.0,NaN,...,1045.47,816.0,NaN,1.17,2728.0,2860.0,3.8,NaN,NaN,NaN
3,2015-01-01T02:00:00Z,2015-01-01T03:00:00+0100,5347.0,6482.0,41.0,NaN,65.0,8799.0,9135.0,NaN,...,1004.79,805.0,NaN,1.04,2626.0,2810.0,3.8,NaN,NaN,NaN
4,2015-01-01T03:00:00Z,2015-01-01T04:00:00+0100,5249.0,6454.0,38.0,NaN,64.0,8567.0,8909.0,NaN,...,983.79,803.0,NaN,1.61,2618.0,2780.0,3.8,NaN,NaN,NaN


In [43]:
long_data = pd.melt(time_data, var_name='variable', value_name='value', id_vars=['utc_timestamp', 'cet_cest_timestamp'])
long_data = long_data.dropna(subset=['value'])
long_data.head()

,utc_timestamp,cet_cest_timestamp,variable,value
1,2015-01-01T00:00:00Z,2015-01-01T01:00:00+0100,AT_load_actual_entsoe_transparency,5946.0
2,2015-01-01T01:00:00Z,2015-01-01T02:00:00+0100,AT_load_actual_entsoe_transparency,5726.0
3,2015-01-01T02:00:00Z,2015-01-01T03:00:00+0100,AT_load_actual_entsoe_transparency,5347.0
4,2015-01-01T03:00:00Z,2015-01-01T04:00:00+0100,AT_load_actual_entsoe_transparency,5249.0
5,2015-01-01T04:00:00Z,2015-01-01T05:00:00+0100,AT_load_actual_entsoe_transparency,5309.0


In [56]:
long_data.head().variable.str.split('_',n=1,expand=True)

,0,1
1,AT,load_actual_entsoe_transparency
2,AT,load_actual_entsoe_transparency
3,AT,load_actual_entsoe_transparency
4,AT,load_actual_entsoe_transparency
5,AT,load_actual_entsoe_transparency


In [57]:
long_data[['country','type']] = long_data.variable.str.split('_', n=1,expand=True)
# long_data['type']  = long_data.variable.str.split('_', expand=True)[1:].str.join('_')
long_data = long_data.drop(columns=['variable'])

long_data.head().to_pickle('long_pickle.pkl')

In [3]:
tmp = pd.read_pickle('long_pickle.pkl')
tmp.head()

,utc_timestamp,cet_cest_timestamp,value,country,type
1,2015-01-01T00:00:00Z,2015-01-01T01:00:00+0100,5946.0,AT,load_actual_entsoe_transparency
2,2015-01-01T01:00:00Z,2015-01-01T02:00:00+0100,5726.0,AT,load_actual_entsoe_transparency
3,2015-01-01T02:00:00Z,2015-01-01T03:00:00+0100,5347.0,AT,load_actual_entsoe_transparency
4,2015-01-01T03:00:00Z,2015-01-01T04:00:00+0100,5249.0,AT,load_actual_entsoe_transparency
5,2015-01-01T04:00:00Z,2015-01-01T05:00:00+0100,5309.0,AT,load_actual_entsoe_transparency


In [4]:
tmp = pd.read_pickle('pv_data.pkl')
tmp.head()

,electricity,irradiance_direct,irradiance_diffuse,temperature,country
1546300800000,0.0,0.0,0.0,4.024,Spain
1546304400000,0.0,0.0,0.0,3.084,Spain
1546308000000,0.0,0.0,0.0,2.221,Spain
1546311600000,0.0,0.0,0.0,1.384,Spain
1546315200000,0.0,0.0,0.0,0.727,Spain
